In [14]:
import tifffile as tif, numpy as np, os, matplotlib.pyplot as plt, sys, pandas as pd
import shutil, itertools 
from collections import Counter
import datetime
from subprocess import check_output
import SimpleITK as sitk

In [ ]:
def point_transformix(pretransform_text_file, transformfile, dst):
    """apply elastix transform to points      
    Inputs
    -------------
    pretransform_text_file = list of points that already have resizing transform
    transformfile = elastix transform file
    dst = folder
    
    Returns
    ---------------
    trnsfrm_out_file = pth to file containing post transformix points
    
    """
    print("Make sure elastix module is loaded!!!")
    sys.stdout.write("\n***********Starting Transformix***********")
    #set paths    
    trnsfrm_out_file = os.path.join(dst, "outputpoints.txt")
    #run transformix point transform
    call = "transformix -def {} -out {} -tp {}".format(pretransform_text_file, dst, transformfile)
    print(check_output(call, shell=True))
    sys.stdout.write("\n   Transformix File Generated: {}".format(trnsfrm_out_file)); sys.stdout.flush()
    return trnsfrm_out_file

In [ ]:
def create_text_file_for_elastix(src, dst):
    """
    Inputs
    ---------
    src = numpy file consiting of nx3 (ZYX points)
    dst = folder location to write points
    """
    print("This function assumes ZYX centers...")
    #setup folder
    if not os.path.exists(dst): os.mkdir(dst)
    #create txt file, with elastix header, then populate points
    pth=os.path.join(dst, "zyx_points_pretransform.txt")
    #load
    
    if type(src) == np.ndarray:
        arr = src
    else:
        print("src param must be a numpy array")

    #convert
    stringtowrite = "\n".join(["\n".join(["{} {} {}".format(i[2], i[1], i[0])]) for i in arr]) ####this step converts from zyx to xyz*****
    #write file
    sys.stdout.write("writing centers to transfomix input points text file..."); sys.stdout.flush()
    with open(pth, "w+") as fl:
        fl.write("index\n{}\n".format(len(arr)))    
        fl.write(stringtowrite)
        fl.close()
    sys.stdout.write("...done writing centers\n"); sys.stdout.flush()
    return pth

In [3]:
#Not used in this script, but can unpack the .txt post-transformed points
def unpack_pnts(points_file, dst=None):
    """
    function to take elastix point transform file and return anatomical locations of those points
    
    Here elastix uses the xyz convention rather than the zyx numpy convention
    
    Inputs
    -----------
    points_file = post_transformed file, XYZ
    
    Returns
    -----------
    dst_fl = path to numpy array, ZYX
    
    """   
    #####inputs 
    assert type(points_file)==str
    point_or_index = 'OutputPoint'
    #get points
    with open(points_file, "r") as f:                
        lines=f.readlines()
        f.close()
    #####populate post-transformed array of contour centers
    sys.stdout.write("\n\n{} points detected\n\n".format(len(lines)))
    #arr=np.empty((len(lines), 3))    
    for i in range(len(lines)):        
        #arr[i,...]=lines[i].split()[lines[i].split().index(point_or_index)+3:lines[i].split().index(point_or_index)+6] #x,y,z            
        lx=lines[i].split()[lines[i].split().index(point_or_index)+3:lines[i].split().index(point_or_index)+6] #x,y,z
        print(lx)
    #optional save out of points
    #dst_fl = os.path.join(dst, "posttransformed_zyx_voxels.npy")
    #np.save(dst_fl, np.asarray([(z,y,x) for x,y,z in arr]))    
    #check to see if any points where found
    #print("output array shape {}".format(arr.shape))   
    #return dst_fl

In [41]:
pfile = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/transformix_output/outputpoints.txt'
unpack_pnts(pfile)



1 points detected

['4322.583151', '1707.856344', '63.256100']


In [ ]:
#Change to path where points are stored. This script is designed for pnts to be a numpy array of coordinates
#in xyz orientation 
pnts = None #load in numpy array of xyz points

#For resizing dimensions. We use the dimensions of the downsized volume that's created as part of 
#our processing pipeline to downsize the points
downsized = None #path to downsized volume
downsized = tif.imread(downsized) #sagittal
zd,yd,xd = downsized.shape #sagittal (numpy's .shape returns the dimensions in z,y,x format)

#reorient pnts to be in zyx format used for elastix
pnts_sag = np.array([[xx[2],xx[1],xx[0]] for xx in pnts])

#We use the dimensions of the fullsize volume to find the downsizing factor 
stitched = None #path to fullsize planes
y,z = tif.imread(os.listdir(stitched)[0]).shape #sagittal, reading in shape of first plane
x = len([xx for xx in os.listdir(stitched) if ".tif" in xx]) #sagittal, number of planes
#Downsizing factor 
f = ((zd/z),(yd/y),(xd/x))
downsized_pnts_sag = np.array([[xx[0]*f[0],xx[1]*f[1],xx[2]*f[2]] for xx in pnts_sag]).astype(int)
            
#transform
#make into transformix-friendly text file
transformed_dst = None #path to where you want the transformed points to be outputted
#Creates text file out of points array to be used for elastix
pretransform_text_file = create_text_file_for_elastix(downsized_pnts_sag, transformed_dst)

#This script assumes elastix has been to obtain transformation parameters
#for the downsized volume -> Allen atlas
elastix_dir = None #path to outputs of elastix transformation for downsized_volume -> Allen atlas
#The TransformParameters.1.txt file that's outputted by elastix includes both the affine and
#non affine transformations
transformfile = os.path.join(elastix_dir, "TransformParameters.1.txt")

#run transformix on points
points_file = point_transformix(pretransform_text_file, transformfile, transformed_dst)
#Points_file is a .txt file of transformed points. The unpack_pnts function can be modified to read in this
#file into an array

In [28]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
fixedfilepath = os.path.join(DATA, 'allen_100um_sagittal.tiff')
movingimagepath = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/aligned_volume.tif'
fixedImage = sitk.ReadImage(fixedfilepath)
movingImage = sitk.ReadImage(movingimagepath)

In [34]:
# Compute the transformation
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(movingImage)
parameter_file_affine = os.path.join(DATA, 'Order1_Par0000affine.debug.txt')
parameter_file_bspline = os.path.join(DATA, 'Order2_Par0000bspline.debug.txt')
elastixImageFilter.SetInitialTransformParameterFileName(parameter_file_affine)
#elastixImageFilter.SetInitialTransformParameterFileName(parameter_file_bspline)
elastixImageFilter.SetOutputDirectory('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/elastix_output')
#parameterMapVector = sitk.VectorOfParameterMap()
#parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
#parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
#elastixImageFilter.SetParameterMap(parameterMapVector)
elastixImageFilter.Execute()

ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/elastix_output/
-threads  unspecified, so all available threads are used
  The default value "true" is used instead.

  From elastix 4.8 it defaults to true!
This may change the behavior of your registrations considerably.

Command line options from TransformBase:
-t0       /net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/Order1_Par0000affine.debug.txt

Reading images...
Reading images took 0 ms.

Reading the elastix parameters from file ...

ERROR: No center of rotation is specified in the transform parameter file

itk::ExceptionObject (0x55c7ed02dbe0)
Location: "unknown" 
File: /src/SimpleElastix/build/Elastix/Components/Transforms/AdvancedAffineTransform/elxAdvancedAffineTransform.hxx
Line: 94
Description: itk::ERROR: AdvancedAffineTrans

RuntimeError: Exception thrown in SimpleITK ElastixImageFilter_Execute: /src/SimpleElastix/Code/Elastix/src/sitkElastixImageFilterImpl.cxx:259:
sitk::ERROR: 
itk::ExceptionObject (0x55c7ed02dbe0)
Location: "unknown" 
File: /src/SimpleElastix/build/Elastix/Core/Main/elxElastixFilter.hxx
Line: 253
Description: itk::ERROR: ElastixFilter(0x55c7e9d98520): Internal elastix error: See elastix log (use LogToConsoleOn() or LogToFileOn()).



In [24]:
# Warp point set. The transformed points will be written to a file named
# outputpoints.txt in the output directory determined by SetOutputDirectory()
# (defaults to working directory). The moving image is needed for transformix
# to correctly infer the dimensionality of the point set.
ptspath = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/com.points.pts'
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetMovingImage(movingImage)
transformixImageFilter.SetTransformParameterMap(elastixImageFilter.GetTransformParameterMap())
transformixImageFilter.SetFixedPointSetFileName(ptspath)
transformixImageFilter.SetOutputDirectory('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/transformix_output')
out = transformixImageFilter.Execute()

Time spent on saving the results, applying the final transform etc.: 16069 ms.
ELASTIX version: 5.000
Command line options from ElastixBase:
-out      /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/transformix_output/
-threads  unspecified, so all available threads are used
-def      /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/com.points.pts
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.000002 s
Calling all ReadFromFile()'s ...
  The default value "false" is used instead.
  The default value "false" is used instead.
  Calling all ReadFromFile()'s took 0.009201 s
Transforming points ...
  The transform is evaluated on some points, specified in the input point file.
  Reading input point file: /net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/com.points.pts
  Input points are specified in world coordinates.
  Number of